In [4]:
pip install numpy pandas cvxpy yfinance scipy scikit-learn



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### req Libraries

In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import cvxpy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

#### 1: Data Collection

In [17]:
def download_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    return data

#### 2: Calculating Returns

In [7]:

def calculate_returns(prices):
    returns = prices.pct_change().dropna()
    return returns



#### 3: Markowitz Portfolio Optimization (Quadratic Programming)

In [8]:
def markowitz_optimization(returns, risk_aversion):
    mu = returns.mean().values
    Sigma = returns.cov().values
    n = len(mu)
    
    w = cp.Variable(n)
    objective = cp.Maximize(mu @ w - risk_aversion * cp.quad_form(w, Sigma))
    constraints = [cp.sum(w) == 1, w >= 0]
    
    prob = cp.Problem(objective, constraints=[cp.sum(w) == 1, w >= 0])
    prob_solution = objective.solve(solver=cp.SCS)
    
    return w.value

In [21]:
pip install cp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cp: filename=cp-2020.12.3-py3-none-any.whl size=1422 sha256=cf481cecb725b6c9c12ec9f0bd62c8f04f3b5263dbf99605ced81ab058a3595f
  Stored in directory: /Users/shuvodeepdutta/Library/Caches/pip/wheels/ce/63/fc/1a421c2c2a8fd4cec74f2b4077b1db348c8f154556b751d2b0
Successfully built cp

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### 4.Dimension Reduction via Linear Programming (LP)

In [22]:
def lp_dimension_reduction(returns):
    mu = returns.mean().values
    A = returns - returns.mean()
    
    n_assets = len(mu)
    
    # LP reformulation (using absolute deviations)
    w = cp.Variable(n_assets)
    v = cp.Variable(A.shape[0])
    
    gamma = 1.0  # penalty parameter
    
    objective = cp.Maximize(gamma * mu @ w - (1/A.shape[0]) * cp.sum(v))
    
    constraints = [
        A.values @ w <= v,
        -A.values @ w <= v,
        cp.sum(w) == 1,
        w >= 0,
        v >= 0
    ]
    
    problem = cp.Problem(objective=objective, constraints=constraints)
    problem.solve(solver=cp.GUROBI)
    
    selected_assets = np.where(w.value > 1e-5)[0]
    
    return selected_assets

#### 5: Sparsification by Correlation Method

In [10]:
def sparsify_covariance(cov_matrix, threshold=0.5):
    corr_matrix = np.corrcoef(cov_matrix)
    
    sparse_cov_matrix = cov_matrix.copy()
    
    # Sparsify covariance matrix based on correlation threshold
    for i in range(len(cov_matrix)):
        for j in range(len(cov_matrix)):
            if i != j and abs(corr_matrix[i,j]) <= threshold:
                cov_matrix[i,j] = 0
    
    # Partial matrix completion to ensure positive semidefiniteness
    eigvals, eigvecs = np.linalg.eigh(cov_matrix)
    
    # Ensure positive semidefinite by adjusting eigenvalues if necessary
    min_eigval = np.min(np.linalg.eigvals(cov_matrix))
    if min(np.linalg.eigvals(cov_matrix)) < 0:
        cov_matrix += np.eye(len(cov_matrix)) * abs(min(np.linalg.eigvals(cov_matrix))) * 1.01
    
    return cov_matrix

#### 6 :Portfolio Optimization using Sparsified Covariance Matrix

In [11]:
def sparse_markowitz_optimization(mu, sparse_cov_matrix, risk_aversion):
    n_assets = len(mu)
    
    w = cp.Variable(n_assets)
    
    objective = cp.Maximize(mu @ w - risk_aversion * cp.quad_form(w, sparse_cov))
    
    constraints = [
        cp.sum(w) == 1,
        w >= 0
    ]
    
    problem = cp.Problem(objective=objective, constraints=constraints)
    problem.solve(solver=cp.GUROBI)
    
    return w.value


#### Examples Usage

In [23]:
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META', 'TSLA', 'JPM', 'JNJ', 'V', 'NVDA']
start_date='2018-01-01'
end_date='2023-12-31'

data = download_stock_data(tickers, start_date='2018-01-01', end_date='2023-12-31')
returns_df = data['Close'].pct_change().dropna()

# Dimension Reduction via LP (predicting assets to keep)
selected_covariance_matrix = lp_dimension_reduction(returns_df)

# Perform portfolio optimization using sparsified covariance matrix with risk aversion parameter λ=10
optimal_weights_sparse_portfolio = sparse_markowitz_optimization(pd.DataFrame(selected_covariance_matrix), risk_aversion=10)

print("Optimal Portfolio Weights (Sparse):")
print(optimal_weights)


[*********************100%***********************]  10 of 10 completed


SolverError: The solver GUROBI is not installed.

#### Explanation of the Code:

1. Data Collection: Historical stock prices are downloaded using yfinance.

2. LP Dimension Reduction: The covariance matrix dimension is reduced by predicting relevant assets through an LP reformulation.

3. Sparsification by Correlation: Covariance entries with weak correlations are set to zero; the resulting matrix is adjusted to maintain positive semidefiniteness.

4. Portfolio Optimization: The Markowitz optimization is performed with the reduced/sparsified covariance matrix.